In [4]:
import sys,os
import trimesh as tri
import numpy as np
import torch
import matplotlib.pyplot as plt
import polyscope as ps
ps.init()

from utils import mesh_processing, lddmm_utils

In [5]:
root_path = "../datasets/COMA/"
root_path_remeshed = "../datasets/COMA_remeshed/"

idents = ['FaceTalk_170725_00137_TA', 'FaceTalk_170728_03272_TA', 'FaceTalk_170731_00024_TA', 'FaceTalk_170809_00138_TA',
          'FaceTalk_170811_03274_TA', 'FaceTalk_170811_03275_TA', 'FaceTalk_170904_00128_TA', 'FaceTalk_170904_03276_TA',
          'FaceTalk_170908_03277_TA', 'FaceTalk_170912_03278_TA', 'FaceTalk_170913_03279_TA', 'FaceTalk_170915_00223_TA']

exprs = ['bareteeth', 'cheeks_in', 'eyebrow', 'high_smile', 'lips_back', 'lips_up',
         'mouth_down', 'mouth_extreme', 'mouth_middle', 'mouth_open', 'mouth_side', 'mouth_up']

In [6]:
def load(root_path, ident, expr, neutral=False):
    path_id_expr = root_path + ident + '/' + expr
    list_id_expr = os.listdir(path_id_expr)
    list_id_expr.sort()
    if neutral:
        return(tri.load(path_id_expr + '/' + list_id_expr[0]))
    
    return(tri.load(path_id_expr + '/' + list_id_expr[len(list_id_expr)//2]))

In [142]:
target_path = '../datasets/results_FLAME/cheeks_in/cheeks_in.000001/scan_scaled.obj'

pred_path = '../Data/COMA/results_identity/pointnet_original_autoencoder/latent_128/train_half/viz_predictions/tst000.ply'
#pred_path = '../datasets/results_FLAME/cheeks_in/cheeks_in.000001/fit_scan_result.obj'

mesh1 = tri.load(target_path)
mesh2 = tri.load(pred_path)
V1, V2 = np.array(mesh1.vertices), np.array(mesh2.vertices)
F1, F2 = np.array(mesh1.faces), np.array(mesh2.faces)
C1, C2 = np.array(mesh1.triangles_center), np.array(mesh2.triangles_center)

In [143]:
import matplotlib as mpl
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

viridis = mpl.colormaps['viridis']

In [144]:
def closest(points, P):
    """
        points: 3xn array representing the point cloud
        P: centre point to measure distance from
        
        Returns: tuple(closest point, distance between P and closest point)
    """ 
    distances = [np.linalg.norm(p) for p in (points - P)]
    idx_of_min = np.argmin(distances)
    
    return (points[idx_of_min], distances[idx_of_min])

In [145]:
def distance_mat_from_ref(V, ref):
    
    return np.array([closest(V,ref[i])[1] for i in range(ref.shape[0])])

In [146]:
D = np.linalg.norm(V2 - V1, axis=1)

In [147]:
D_faces = np.linalg.norm(C2 - C1, axis=1)

In [148]:
D.max()

0.0219465288623515

In [149]:
ps_mesh = ps.register_surface_mesh("my mesh", V1, F1)

# visualize some random data per-vertex
ps_mesh.add_scalar_quantity("rand vals", D, enabled=True, vminmax=(0,0.02), cmap='jet')

# visualize some random data per-face
ps_mesh.add_scalar_quantity("rand vals2", D_faces, defined_on='faces', vminmax=(0,0.02), cmap='jet')

# view the mesh with all of these quantities
ps.show() 